<img src="https://filedn.com/l31Uxc2iCI1koQr1EKWjwQH/Research/gee_recharge-model/hydrogeologygroup.png" alt="Hydrogeology Group" width="200"/>

# Regional Recharge Estimation

Authors: Elizabeth Guzman-Hidalgo, Saul Arciniega-Esparza

Faculty of Engineering at the National Autonomous University of Mexico


## 1. Authenticate Google Earth Engine project
You need a Google Cloud account and a project linked to Google Earth Engine for non-commercial use. You can follow [this tutorial](https://www.youtube.com/watch?v=-kUuQF_D4Jw) if you don't have an active project:
## 2. Install requirements and load libraries

This section installs the required libraries and loads the visualization packages needed to run the potential recharge model in Google Earth Engine. Functions are defined for the acquisition and preprocessing of input variables, including soil texture properties and climatological data on precipitation and potential evapotranspiration. The functions implementing the Thornthwaite-Mather water balance model are then defined to generate a new dataset at a monthly temporal scale, with bands corresponding to potential recharge (rech), precipitation (pr), potential evapotranspiration (pet), accumulated potential water loss (apwl), and soil water storage at field capacity (st).

## 3. Run application
This section implements an interactive panel in which users can define the following parameters:

**Start and End year:**
Allows the simulation period to be defined with a maximum duration of five years. The sliders are dynamically linked to prevent exceeding this range, thereby restricting the selection of longer simulation periods.

**Root depth:**
Allows the definition of an average rooting depth, in meters, representative of the study area.

**PET bias factor:**
Allows the definition of a correction factor applied to the MODIS potential evapotranspiration dataset, enabling the adjustment (increase or decrease) of PET values in cases where systematic overestimation or underestimation relative to reference data or field observations has been identified within the study area.

**Visualization parameters:**
Allows the definition of the color scale ranges, in millimeters, for the output maps (recharge, precipitation, and PET).

**Select analysis zone:**
Allows the spatial domain of the estimation to be defined, which may correspond to the entire Basin of Mexico, a specific aquifer within the basin, or, if left blank, a global estimation is produced.

**Run model:**
Generates spatial maps of Precipitation, PET (potential evapotranspiration), and Recharge for the previously selected area. If a simulation area is specified in the "Select analysis zone" field, the model also produces monthly time series for that area; otherwise, only global maps are generated.

**Download Series:**
Enables the export of the simulated time series in CSV format for the selected simulation area.

In [ ]:
# @title 1. Google Earth Engine User authentication

Google_Cloud_Project_ID = ""  # @param {type:"string"}

import ee
import geemap

ee.Authenticate()
ee.Initialize(project=Google_Cloud_Project_ID)
geemap.ee_initialize()

In [ ]:
# @title 2. Install requirements and load libraries
%%capture
!pip install PyCRS
!pip install hvplot

import codecs
import base64
import json
import requests
from datetime import datetime
import pandas as pd
import hvplot
import hvplot.pandas
import numpy as np
import folium
import pprint
import branca.colormap as cm
import ipywidgets as widgets
from ipywidgets import AppLayout, Layout
from IPython.display import display, Javascript


def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# ============================================================
# Soil properties processing
# ============================================================

# Soil depths (cm) and their corresponding OpenLandMap
# band identifiers (b0, b10, b30, b60, b100, b200)
olm_depths = [0, 10, 30, 60, 100, 200]
olm_bands = ["b" + str(sd) for sd in olm_depths]

def get_soil_prop(param, region=None):
    """
    This function returns soil properties image
    param (str): must be one of:
        "sand"     - Sand fraction [%w]
        "clay"     - Clay fraction [%w]
        "orgc"     - Organic Carbon fraction [g/kg]
    Scale factors are defined in accordance with
    each dataset description
    """
    if param == "sand":
        snippet = "OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02"
        scale_factor = 1 * 0.01
    elif param == "clay":
        snippet = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
        scale_factor = 1 * 0.01
    elif param == "orgc":
        snippet = "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
        scale_factor = 5 * 0.001
    else:
        return print("error")

    dataset = ee.Image(snippet).multiply(scale_factor)

    if region:  # Limiting the study area avoids global calculations
       dataset = dataset.clip(region)
    return dataset


def compute_field_capacity(sand, clay, orgc):
    """
    Computes soil wilting point (θ1500) and field capacity (θ33) at standard
    soil depths using Saxton–Rawls pedotransfer functions based on soil texture
    and organic matter.
    """
    # Initialize output images
    wilting_point = ee.Image(0)
    field_capacity = ee.Image(0)
    # Conversion from organic carbon to organic matter (Van Bemmelen factor)
    orgm = orgc.multiply(1.724)
    # Compute properties for each reference soil depth
    for key in olm_bands:
        si = sand.select(key)
        ci = clay.select(key)
        oi = orgm.select(key)

        # Compute theta_1500t parameter
        theta_1500ti = (
            ee.Image(0)
            .expression(
                "-0.024 * S + 0.487 * C + 0.006 * OM + 0.005 * (S * OM)\
            - 0.013 * (C * OM) + 0.068 * (S * C) + 0.031",
                {
                    "S": si,
                    "C": ci,
                    "OM": oi,
                },
            )
            .rename("T1500ti")
        )
        # Final wilting point expresion
        wpi = theta_1500ti.expression(
            "T1500ti + ( 0.14 * T1500ti - 0.002)", {"T1500ti": theta_1500ti}
        ).rename("wpi")

        # Add as a new band in the wilting point ee.Image
        wilting_point = wilting_point.addBands(wpi.rename(key).float())


        # Compute theta_33t parameter
        theta_33ti = (
            ee.Image(0)
            .expression(
                "-0.251 * S + 0.195 * C + 0.011 * OM +\
            0.006 * (S * OM) - 0.027 * (C * OM)+\
            0.452 * (S * C) + 0.299",
                {
                    "S": si,
                    "C": ci,
                    "OM": oi,
                },
            )
            .rename("T33ti")
        )
        # Final soil field capacity expression
        fci = theta_33ti.expression(
            "T33ti + (1.283 * T33ti * T33ti - 0.374 * T33ti - 0.015)",
            {"T33ti": theta_33ti.select("T33ti")},
        )
        # Add as a new band in the wilting point ee.Image
        field_capacity = field_capacity.addBands(fci.rename(key).float())

    return wilting_point, field_capacity


def olm_prop_mean(olm_image, band_output_name):
    """
    Computes the arithmetic mean of a soil property across
    standard discrete depth intervals (0, 10, 30, 60, 100,
    and 200 cm) from OpenLandMap layers.
    """
    mean_image = olm_image.expression(
        "(b0 + b10 + b30 + b60 + b100 + b200) / 6",
        {
            "b0": olm_image.select("b0"),
            "b10": olm_image.select("b10"),
            "b30": olm_image.select("b30"),
            "b60": olm_image.select("b60"),
            "b100": olm_image.select("b100"),
            "b200": olm_image.select("b200"),
        },
    ).rename(band_output_name)
    return mean_image

# ============================================================
# Climatic datasets processing
# ============================================================

def sum_resampler(coll, freq, unit, scale_factor, band_name):
    """
    Resamples an ee.ImageCollection to a defined temporal resolution.

    Parameters:
    -----------
    coll : ee.ImageCollection name
    freq : (int) Resampling frequency.
    unit : (str) Resampling time unit ('day', 'month', or 'year').
    scale_factor : (float) Scale factor for the band values.
    band_name : (str) Name of the output band.

    For each period defined by 'freq' and 'unit', the function computes
    the mean of the selected band, multiplies by the period
    length and a scale factor, and returns an ee.ImageCollection
    with the averaged band.
    """

    # Define initial and final dates of the collection.
    firstdate = ee.Date(
        coll.sort("system:time_start", True).first().get("system:time_start")
    )
    lastdate = ee.Date(
        coll.sort("system:time_start", False).first().get("system:time_start")
    )
    # Compute time period of the collection
    diff_dates = lastdate.difference(firstdate, unit)
    # Create a new time index for resampling
    new_index = ee.List.sequence(0, ee.Number(diff_dates), freq)

    # Function to apply to the new time index
    def apply_resampling(date_index):
        # Get start and end dates for the period
        startdate = firstdate.advance(ee.Number(date_index), unit)
        enddate = firstdate.advance(ee.Number(date_index).add(freq), unit)
        # Compute period length in days
        diff_days = enddate.difference(startdate, "day")
        # Compute average composite image for the period
        image = (
            coll.filterDate(startdate, enddate)
            .mean()
            .multiply(diff_days)
            .multiply(scale_factor)
            .rename(band_name)
        )
        return image.set("system:time_start", startdate.millis())
    # Apply the resampling function
    res = new_index.map(apply_resampling)
    res = ee.ImageCollection(res)
    return res

# ============================================================
# Thornthwaite–Mather recharge model
# ============================================================

def recharge_model(precipitation, pet, sand, clay, orgc, rooting_depth=0.1, region=None):
    """
    Computes potential groundwater recharge using the Thornthwaite–Mather water balance model.

    Parameters
    ----------
    precipitation : ee.ImageCollection
    pet : ee.ImageCollection
    sand : ee.Image
    clay : ee.Image
    orgc : ee.Image
    rooting_depth : float
    region : ee.Geometry

    Returns
    -------
    ee.ImageCollection with bands:
    - 'rech': potential groundwater recharge [mm]
    - 'apwl': acumulated potential water loss [mm]
    - 'st': soil water storage [mm]
    - 'pr': precipitation [mm]
    - 'pet': potential evapotranspiration [mm]
    """

    # Merge precipitation and PET into a single collection sorted by date
    meteo = precipitation.combine(pet).sort("system:time_start")
    #Compute wilting point and field capacity and average across soil depths
    wilting_point, field_capacity = compute_field_capacity(sand, clay, orgc)
    fcm = olm_prop_mean(field_capacity, "fc_mean")
    wpm = olm_prop_mean(wilting_point, "wp_mean")
    # Enter the average rooting depth for the study area
    zr = ee.Image(rooting_depth)
    p = ee.Image(0.5)
    # Compute total available water (TAW) and soil water storage at field capacity (STfc)
    taw = ((fcm.select("fc_mean").subtract(wpm.select("wp_mean"))).multiply(1000).multiply(zr))
    stfc = taw.multiply(p)
    # Identify urban areas from land cover dataset to restrict recharge
    landcover = ee.ImageCollection('MODIS/061/MCD12Q1').select('LC_Type1') \
                 .filterDate("2022-01-01", "2022-12-31").first()
    if region:  # Limiting the study area to avoid global calculations
       landcover = landcover.clip(region)
    urban_area_mask = landcover.eq(13)


    # Model initialization
    time0 = ee.Date(meteo.first().get("system:time_start"))
    initial_rech = ee.Image(0).set("system:time_start", time0).select([0], ["rech"]).float()
    initial_apwl = ee.Image(0).set("system:time_start", time0).select([0], ["apwl"]).float()
    initial_st = stfc.set("system:time_start", time0).select([0], ["st"]).float()
    initial_pr = ee.Image(0).set("system:time_start", time0).select([0], ["pr"]).float()
    initial_pet = ee.Image(0).set("system:time_start", time0).select([0], ["pet"]).float()

    initial_image = initial_rech.addBands(ee.Image([initial_apwl, initial_st, initial_pr, initial_pet]))
    image_list = ee.List([initial_image])



    def recharge_calculator(image, image_list):
        """
        Thornthwaite-Mather model calculations at each iteration.
        """
        # Read the timestamp (system:time_start) of the current ee.Image
        localdate = image.date().millis()
        # Import previous image stored in the list.
        prev_im = ee.Image(ee.List(image_list).get(-1))
        # Import previous APWL and ST.
        prev_apwl = prev_im.select("apwl")
        prev_st = prev_im.select("st")
        # Import current precipitation and evapotranspiration.
        pr_im = image.select("pr")
        pet_im = image.select("pet")
        # Initialize the new bands associated with recharge, apwl and st.
        new_rech = (
            ee.Image(0)
            .set("system:time_start", localdate)
            .select([0], ["rech"])
            .float()
        )
        new_apwl = (
            ee.Image(0)
            .set("system:time_start", localdate)
            .select([0], ["apwl"])
            .float()
        )
        new_st = (
            prev_st.set("system:time_start", localdate).select([0], ["st"]).float()
        )
        # Compute bands using boolean masks and logical operations.
        # CASE 1.
        # Define zone1: the area where PET > P.
        zone1 = pet_im.gt(pr_im)
        # Calculation of APWL in zone 1.
        zone1_apwl = prev_apwl.add(pet_im.subtract(pr_im)).rename("apwl")
        # Implementation of zone 1 values for APWL.
        new_apwl = new_apwl.where(zone1, zone1_apwl)
        # Calculate ST in zone 1.
        zone1_st = stfc.multiply(
            ee.Image.exp(zone1_apwl.divide(stfc).multiply(-1))
        ).rename("st")
        # Implement ST in zone 1.
        new_st = new_st.where(zone1, zone1_st)
        # CASE 2.
        # Define zone2: the area where PET <= P.
        zone2 = pet_im.lte(pr_im)
        # Calculate ST in zone 2.
        zone2_st = prev_st.add(pr_im).subtract(pet_im).rename("st")
        # Implement ST in zone 2.
        new_st = new_st.where(zone2, zone2_st)

        # CASE 2.1.
        # Define zone21: the area where PET <= P and ST >= STfc.
        zone21 = zone2.And(zone2_st.gte(stfc))
        # Calculate recharge in zone 21.
        zone21_re = zone2_st.subtract(stfc).rename("rech")
        # Implement recharge in zone 21.
        new_rech = new_rech.where(zone21, zone21_re)
        # Implement ST in zone 21.
        new_st = new_st.where(zone21, stfc)
        # CASE 2.2.
        # Define zone 22: the area where PET <= P and ST < STfc.
        zone22 = zone2.And(zone2_st.lt(stfc))
        # Calculate APWL in zone 22.
        zone22_apwl = (
            stfc.multiply(-1).multiply(ee.Image.log(zone2_st.divide(stfc))).rename("apwl")
        )
        # Implement APWL in zone 22.
        new_apwl = new_apwl.where(zone22, zone22_apwl)
        # Create a spatial mask where recharge can effectively be calculated.
        # Where PET, P, FCm, and WPm are valid, excluding urban areas.
        new_rech = new_rech.where(urban_area_mask, 0)
        # Add all computed bands to the output ee.Image.
        new_image = new_rech.addBands(ee.Image([new_apwl, new_st, pr_im, pet_im]))
        # Add the new ee.Image to the iteration list ee.List.
        return ee.List(image_list).add(new_image)
    # Iterate the recharge function over the meteo ImageCollection.
    rech_list = meteo.iterate(recharge_calculator, image_list)
    # Remove the initial image used to initialize the model from the list.
    rech_list = ee.List(rech_list).remove(initial_image)
    # Transform the list into an ee.ImageCollection.
    rech_collection = ee.ImageCollection(rech_list)

    return rech_collection


def correct_pet(image, factor=1.0):
    """
    Applies a multiplicative bias-correction factor to PET images.

    """
    date = ee.Date(image.get('system:time_start'))
    img_corrected = image.multiply(factor).float()
    return img_corrected.copyProperties(image, ['system:time_start'])


def run_recharge_model(start, end, rooting_depth=0.1, pet_fbias=1.0, region=None):
    """
    Runs the recharge simulation using the Thornthwaite–Mather model
    """

    # Soil properties preparation
    sand = get_soil_prop("sand", region)
    clay = get_soil_prop("clay", region)
    orgc = get_soil_prop("orgc", region)
    # Load and filter meteorological datasets from Google Earth Engine
    precipitation = (
        ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
        .select("precipitation")
        .filterDate(start, end)
    )
    pet = (
        ee.ImageCollection("MODIS/061/MOD16A2GF")
        .select("PET")
        .filterDate(start, end)
    )
    # Meteorological data resampling and PET correction
    Precipitation_monthly = sum_resampler(precipitation, 1, "month", 1, "pr")
    PET_monthly = sum_resampler(pet, 1, "month", 0.0125, "pet")
    PET_monthly_corrected = PET_monthly.map(lambda image: correct_pet(image, factor=pet_fbias))
    # Run recharge model
    simulation = recharge_model(Precipitation_monthly, PET_monthly_corrected, sand, clay, orgc, rooting_depth=rooting_depth)
    if region:
        simulation = simulation.map(lambda img: img.clip(region))
    return simulation


def show_map(prec, pet, rg, rg_minmax=None, prec_minmax=None, pet_minmax=None):
    """
    Generate an interactive map for precipitation, PET, and groundwater recharge
    """
    # Viz parameters
    if rg_minmax:
        vmin1 = rg_minmax[0]
        vmax1 = rg_minmax[1]
    else:
        vmin1 = 0
        vmax1 = 300
    if prec_minmax:
        vmin2 = prec_minmax[0]
        vmax2 = prec_minmax[1]
    else:
        vmin2 = 200
        vmax2 = 1500
    if pet_minmax:
        vmin3 = pet_minmax[0]
        vmax3 = pet_minmax[1]
    else:
        vmin3 = 1900
        vmax3 = 2300

    rech_vis_params = {
      "bands": "rech",
      "min": vmin1,
      "max": vmax1,
      "opacity": 1,
      "palette": ["red", "orange", "yellow", "green", "blue", "purple"],
    }
    pr_vis_params = {
        "bands": "pr",
        "min": vmin2,
        "max": vmax2,
        "opacity": 1,
        "palette": ["#440154", "#3B528B", "#21918C", "#5DC863", "#FDE725"],
    }
    pet_vis_params = {
        "bands": "pet",
        "min": vmin3,
        "max": vmax3,
        "opacity": 1,
        "palette": ["#fc9272", "#fcbba1",  "#fed976", "#a1d99b", "#9ecae1"],
    }
    Map = geemap.Map(center=[19.432924, -98.638534], zoom=9)
    # Add maps
    Map.addLayer(prec, pr_vis_params, "Precipitation", False)
    Map.addLayer(pet, pet_vis_params, "PET", False)
    Map.addLayer(rg, rech_vis_params, "Recharge", True)
    # Add color bars
    Map.add_colorbar(vis_params=rech_vis_params, label="Mean Annual Recharge (mm/year)", position="bottomright")
    Map.add_colorbar(vis_params=pr_vis_params, label="Mean Annual Precipitation (mm/year)", position="bottomright")
    Map.add_colorbar(vis_params=pet_vis_params, label="Mean Annual PET (mm/year)", position="bottomright")
    # Add inspection tools
    Map.add_inspector(position='bottomleft')
    # Add layer controls
    Map.add_layer_control()
    return Map


def run_model_widget(start, end, rooting_depth=0.1, pet_fbias=1.0, region=None,
                     rg_minmax=None, prec_minmax=None, pet_minmax=None):
    """
    Execute the recharge model and visualize annual results
    """
    simulation = run_recharge_model(
        start,
        end,
        rooting_depth=rooting_depth,
        pet_fbias=pet_fbias,
        region=region
    )
    # Mean annual values
    rg = simulation.select("rech").mean().multiply(12)
    prec = simulation.select("pr").mean().multiply(12)
    pet = simulation.select("pet").mean().multiply(12)
    Map = show_map(prec, pet, rg, rg_minmax, prec_minmax, pet_minmax)
    return Map, simulation


def zonal_time_series(collection, geometry, scale=5000):
    """
    Compute zonal mean time series over a given geometry.
    """
    def per_img(img):
            stats = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=scale,
                maxPixels=1e13
            )
            return ee.Feature(None, stats).set({
                "date": img.date().format("YYYY-MM"),
            })

    feature = collection.map(per_img)
    data = feature.getInfo()["features"]
    raw_data = []
    for raw in data:
        raw_data.append(raw["properties"])
    data_df = pd.DataFrame(raw_data).set_index("date")
    data_df.index = pd.to_datetime(data_df.index)
    data_df.rename(
        columns={
            "pr": "Precipitation (mm)",
            "pet": "PET (mm)",
            "rech": "Recharge (mm)",
            "apwl": "APWL (mm)",
            "st": "Storage (mm)",
        },
        inplace=True
    )
    return data_df


#%%Load study area geometries from a remote JSON and convert them to ee.Geometry objects
remote_file_geometries = "https://filedn.com/l31Uxc2iCI1koQr1EKWjwQH/Research/gee_recharge-model/recharge-model_geometries.json"
response = requests.get(remote_file_geometries)
geometries = response.json()
ee_geometries = {key: ee.Geometry.Polygon(geometries[key]["geometry"]) for key in geometries.keys()}

In [ ]:
# @title 3. Run application

#%% Widgets functions
def run_command(b):
    output.clear_output()
    output_plot.clear_output()
    button_runmodel.description = "Running..."
    button_runmodel.button_style = "warning"
    button_runmodel.disabled = True
    button_download.disabled = True

    dstart = pd.to_datetime(f"{widget_start_year.value}-01-01")
    dend = pd.to_datetime(f"{widget_end_year.value}-12-31")
    val_zr = widget_zr.value
    val_fr = widget_fr.value
    viz_rech = [widget_viz_min_rech.value, widget_viz_max_rech.value]
    viz_prec = [widget_viz_min_prec.value, widget_viz_max_prec.value]
    viz_pet = [widget_viz_min_pet.value, widget_viz_max_pet.value]
    zone = widget_selection.value
    ee_layer = ee_geometries.get(zone, None)

    with output:
        display([dstart, dend, val_zr, val_fr, ee_layer])

    if dstart > dend:
        with output:
            display("❌ Error: The initial date cannot be later than the final date.")
    else:
        output.clear_output()
        output_plot.clear_output()
        try:
            map, simulation = run_model_widget(
                dstart.strftime("%Y-%m-%d"),
                dend.strftime("%Y-%m-%d"),
                rooting_depth=val_zr,
                pet_fbias=val_fr,
                region=ee_layer,
                rg_minmax=viz_rech,
                prec_minmax=viz_prec,
                pet_minmax=viz_pet
            )
            with output:
                display(map)
            if ee_layer:
                ts = zonal_time_series(simulation, ee_layer, scale=5000)

                def download_csv(b):
                    csv = ts.round(4).to_csv()
                    b64 = base64.b64encode(csv.encode()).decode()
                    payload = f'''
                    var link = document.createElement('a');
                    link.href = "data:text/csv;base64,{b64}";
                    link.download = "simulation_series.csv";
                    document.body.appendChild(link);
                    link.click();
                    document.body.removeChild(link);
                    '''
                    display(Javascript(payload))

                button_download.on_click(download_csv)
                button_download.disabled = False

                plot_figure = ts.hvplot.line(
                    y=["Precipitation (mm)","PET (mm)" , "Recharge (mm)"],
                    legend='top',
                    title="legend=top",
                    responsive=True,
                    min_height=400
                )
                with output_plot:
                    display(plot_figure)
        except:
            with output:
                display("❌ Error: Try to change the simulation period or the model parameters.")

    button_runmodel.description = "Run model"
    button_runmodel.button_style = "success"
    button_runmodel.disabled = False


def handle_slider_year(change):
    widget_start_year.value = min(widget_start_year.value, widget_end_year.value)
    widget_end_year.value = max(widget_start_year.value, widget_end_year.value)
    if int(widget_end_year.value - widget_start_year.value) > 5:
        widget_end_year.value = widget_start_year.value + 5


#%% Widgets
style = {"description_width": "initial"}
widget_start_year = widgets.IntSlider(
    value=datetime.now().year-6,
    min=2000,
    max=datetime.now().year-1,
    step=1,
    description="Start year:",
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style=style
)
widget_end_year = widgets.IntSlider(
    value=datetime.now().year-1,
    min=2000,
    max=datetime.now().year-1,
    step=1,
    description="End year:",
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style=style
)
widget_zr = widgets.FloatText(
    value=0.1,
    description="Root depth (m):",
    step=0.01,
    disabled=False,
    style=style
)
widget_fr = widgets.FloatText(
    value=1.0,
    description="PET bias factor:",
    step=0.01,
    disabled=False,
    style=style
)
widget_viz_min_rech = widgets.FloatText(
    value=0,
    description="Map min Recharge:",
    step=1,
    disabled=False,
    style=style
)
widget_viz_max_rech = widgets.FloatText(
    value=300,
    description="Map max Recharge:",
    step=1,
    disabled=False,
    style=style
)
widget_viz_min_prec = widgets.FloatText(
    value=200,
    description="Map min Precipitation:",
    step=1,
    disabled=False,
    style=style
)
widget_viz_max_prec = widgets.FloatText(
    value=1500,
    description="Map max Precipitation:",
    step=1,
    disabled=False,
    style=style
)
widget_viz_min_pet = widgets.FloatText(
    value=1900,
    description="Map min PET:",
    step=1,
    disabled=False,
    style=style
)
widget_viz_max_pet = widgets.FloatText(
    value=2300,
    description="Map max PET:",
    step=1,
    disabled=False,
    style=style
)
button_runmodel = widgets.Button(
    description="Run model",
    button_style="success",
    tooltip="Clic to compute",
    style=style
)
button_download = widgets.Button(
    description='Download Series',
    icon='download',
    disabled=True
)
output = widgets.Output()
output_plot = widgets.Output()
widget_selection = widgets.Dropdown(
    options=[""] + list(geometries.keys()),
    value="",
    description="Select analysis zone:",
    style=style
)

#%% Link functions
widget_start_year.observe(handle_slider_year, names="value")
widget_end_year.observe(handle_slider_year, names="value")
button_runmodel.on_click(run_command)

#%% App
ui = widgets.VBox([
    widgets.Label(value="Recharge Model parameters"),
    widgets.HBox([widget_start_year, widget_end_year]),
    widgets.HBox([widget_zr, widget_fr]),
    widgets.Label(value="Visualization parameters"),
    widgets.HBox([widget_viz_min_rech, widget_viz_max_rech]),
    widgets.HBox([widget_viz_min_prec, widget_viz_max_prec]),
    widgets.HBox([widget_viz_min_pet, widget_viz_max_pet]),
    widgets.Label(value="Run the model"),
    widget_selection,
    widgets.HBox([button_runmodel, button_download]),
    output,
    output_plot
])

display(ui)